# 作业5

### 第1题

利用 PySpark 实现一个分布式估计圆周率 $\pi$ 的程序，原理如下：

在正方形 $\{(x,y):-1\le x \le 1, -1\le y \le 1\}$ 中随机生成 $N$ 个独立的均匀分布随机数 $(X_i,Y_i)$，其中每个点 $(X_i,Y_i)$ 落入圆 $R=\{(x,y): x^2+y^2\le 1\}$ 的概率是 $\pi/4$。因此，如果随机生成的 $N$ 个点中有 $n$ 个落入圆 $R$ 中，那么 $\pi$ 的估计就是 $4n/N$。

![](https://media.geeksforgeeks.org/wp-content/uploads/MonteCarlo.png)

现在我们采用分布式的方法并行模拟大量的随机数。考虑将所有的点分成100组，每组生成10000个点，每组独立产生随机数并计算落入圆内的数量，最后将所有100组的结果汇总并得出最终 $\pi$ 的估计。为了使结果可重复，第 $i$ 组在生成随机数时使用 $i$ 作为随机数种子。PySpark 使用本地模式，开启 8 个 CPU 核心。

**提示**：使用标准方法启动 PySpark 后，可以利用 `sc.parallelize()` 从一个迭代器或列表生成 RDD，如 `sc.parallelize(range(10))` 和 `sc.parallelize([1, 2, 3])`。

In [ ]:
# 此处插入代码

### 第2题

在 `lec12-admm-lasso.ipynb` 的基础上，利用 ADMM 算法求解 Lasso 问题

$$\frac{1}{2}\Vert y-X\beta\Vert^2+\lambda \Vert \beta\Vert_1,$$

并将其封装成一个函数：

```python
admm_lasso(X, y, lam, rho=1.0, maxit=10000, eps=1e-3, verbose=0)
```

1. 其中 `X` 是 $n\times p$ 的自变量矩阵，`y` 是 $n\times 1$ 的因变量向量，`lam` 是惩罚项参数 $\lambda$，`rho` 是 ADMM 算法的 $\rho$ 参数，`maxit` 是最大迭代次数，`eps` 是 ADMM 收敛的残差临界值，`verbose` 表示是否输出迭代信息，如果 $>0$，则每隔 1000 次迭代打印出当前的两类残差，如果 $\le 0$ 否则不输出任何信息。
2. 参考 `lec12-admm-lad.ipynb` 中的 Cholesky 分解方法，只对矩阵进行一次分解，从而在每次迭代中高效地求解线性方程组。
3. 函数需返回两个量，第一个表示实际使用的迭代次数，第二个表示估计的回归系数。

In [ ]:
# 此处插入代码

def admm_lasso(X, y, lam, rho=1.0, maxit=10000, eps=1e-3, verbose=0):
    # 此处插入代码

利用模拟训练集数据测试上述编写的函数：

In [ ]:
np.random.seed(123)
n = 1000
p = 30
nz = 20
Xtrain = np.random.normal(size=(n, p))
# 真实的 x 只有前20个元素非零，其余均为0
beta = np.random.normal(size=nz)
beta = np.concatenate((beta, np.zeros(p - nz)))
ytrain = Xtrain.dot(beta) + np.random.normal(size=n)
beta

In [ ]:
admm_lasso(Xtrain, ytrain, lam=0.1 * n, maxit=10000, eps=1e-3, verbose=1)

In [ ]:
admm_lasso(Xtrain, ytrain, lam=0.01 * n, maxit=10000, eps=1e-3, verbose=0)

### 第3题

利用第2题中编写的函数，对一个新的测试集数据做预测。首先生成模拟数据：

In [ ]:
np.random.seed(123)
ntest = 500
p = 30
Xtest = np.random.normal(size=(ntest, p))
ytest = Xtest.dot(beta) + np.random.normal(size=ntest)

取 $\lambda=0.1 n$，利用训练集估计回归系数，然后对测试集的因变量做预测，计算预测结果的均方误差，即
$$
MSE=\frac{1}{n_{test}}\sum_{i=1}^{n_{test}}(\hat{y}_i-y_i)^2,
$$
其中 $y_i$ 是第 $i$ 个测试集观测的因变量取值，$\hat{y}_i=x_i'\hat{\beta}$ 是第 $i$ 个观测的因变量预测值。

In [ ]:
# 此处插入代码

利用 PySpark 来并行地对 Lasso 模型的 $\lambda$ 参数进行调优，并考察 $\rho$ 参数对算法收敛速度的影响。取 $\rho=0.1,0.2,\ldots,1.0$，$\lambda=0.1n,0.01n,0.001n$。对于 $\rho$ 和 $\lambda$ 的这 30 个组合，分别利用训练集拟合 Lasso 模型，返回迭代次数，并计算在测试集上的预测 MSE。最终输出如下的结果：

```
rho = 0.1, lambda/n = 0.1, niter = ..., mse = ...
rho = 0.1, lambda/n = 0.01, niter = ..., mse = ...
...
```

**提示**：先生成 $\rho$ 和 $\lambda$ 所有组合的列表，类似于 `params = [(0.1, 0.1), (0.1, 0.01), (0.1, 0.001), (0.2, 0.1), ...]`，然后利用 `sc.parallelize(params)` 生成一个 RDD，最后对这个 RDD 进行 `map()` 和 `collect()` 操作。

In [ ]:
# 此处插入代码